In [2]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree._tree import TREE_LEAF
from sklearn.model_selection import GridSearchCV, train_test_split
from tqdm import tqdm

In [10]:
taiwan_df = pd.read_csv('taiwan-original.csv')
taiwan_df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [11]:
# drop ID row as it is meaningless
taiwan_df = taiwan_df.drop(columns=["ID"])

In [12]:
# separate labels
taiwan_y = taiwan_df["default payment next month"]
taiwan_X = taiwan_df.drop("default payment next month", axis=1)

In [13]:
taiwan_age = taiwan_X["AGE"]
taiwan_sex = taiwan_X["SEX"]
taiwan_X = taiwan_X.drop(columns=["AGE", "SEX"])

In [16]:
# make a train and test split with same proportional size as Adult dataset 
adult_test_size = 1/3
taiwan_train_X, taiwan_test_X, taiwan_train_y, taiwan_test_y = train_test_split(taiwan_X, taiwan_y, test_size=adult_test_size, random_state=42)

# also for the sensitive attributes with same random_state
taiwan_train_sex, taiwan_test_sex, taiwan_train_y, taiwan_test_y = train_test_split(taiwan_sex, taiwan_y, test_size=adult_test_size, random_state=42)
taiwan_train_age, taiwan_test_age, taiwan_train_y, taiwan_test_y = train_test_split(taiwan_age, taiwan_y, test_size=adult_test_size, random_state=42)

In [18]:
# reindex all datasets
taiwan_train_X = taiwan_train_X.reset_index(drop=True)
taiwan_train_y = taiwan_train_y.reset_index(drop=True)
taiwan_train_sex = taiwan_train_sex.reset_index(drop=True)
taiwan_train_age = taiwan_train_age.reset_index(drop=True)

taiwan_test_X = taiwan_test_X.reset_index(drop=True)
taiwan_test_y = taiwan_test_y.reset_index(drop=True)
taiwan_test_sex = taiwan_test_sex.reset_index(drop=True)
taiwan_test_age = taiwan_test_age.reset_index(drop=True)

## PAFER
The code that implements the PAFER algorithm as found in Algorithm 1 in the paper https://arxiv.org/abs/2312.08413

In [20]:
def oracle(dataset, sens_dataset, rule, s_i, mechanism=None, epsilon=0.05, delta=0.001):
    """Returns some (differentially privatised) statistics on the sensitive attribute for the specified dataframe and rule.
    
    Args:
        dataset: The DataFrame that the developers own, which does not contain sensitive attributes.
            Used to calculate total quantities in (root) nodes.
        sens_dataset: A Series that the developers do not own, which contains the sensitive attributes. 
            Combined sensitive attributes should be encoded as a Series, e.g. Black-Female
        rule: The rule for which the to estimate the sensitive attribute. 
            rule must be a pandas conditional expression as a string, e.g. "(adult_test_cat_X['marital-status_Married-civ-spouse']<= 0.5)"
        s_i: The sensitive attribute, its name comes from the ith element in the set S of sensitive attributes.
            s_i should thus be in sens_dataset. 
        mechanism: The privacy mechanism used on the returned counts. Can be one of "gaussian", "laplacian", "exponential", None. 
        epsilon: The privacy budget. Should be larger than 0.
        delta: The privacy margin. Ignored when mechanism is either laplacian or gaussian. Should be in (0, 1]. 
        
    Returns:
        The number of times s_i occurs in sens_dataset, potentially privatised via the mechanism. 
        """
        
    # check epsilon and delta parameters
    if epsilon <= 0 or (mechanism == "gaussian" and (delta <= 0 or delta > 1 or epsilon > 1)):
        raise ValueError("The value of delta should be in (0,1] when using the gaussian mechanism")
    
    if not sens_dataset.isin([s_i]).any():
        raise KeyError("The requested sensitive attribute (s_i) is not in the sensitive dataframe (sens_dataset)")
        
    # the answer if no privacy mechanism is applied
    try:
        # engine might differ for your version, i.e. engine="pandas"
        no_mechanism = sens_dataset.loc[dataset[pd.eval(rule, engine='python')].index].value_counts(sort=False)[s_i]
        
    except KeyError:
        no_mechanism = 0
    
    if mechanism == "laplacian":
        # this is a histogram query so the l1-sensitivity = 1 as per Dwork & Roth 
        sensitivity = 1
        return no_mechanism + np.random.laplace(loc=0, scale=sensitivity / epsilon)
    
    elif mechanism == "gaussian":
        # this is a histogram query so the l2-sensitivity = 2 as per Dwork & Roth
        sensitivity = 2
        return no_mechanism + np.random.normal(loc=0, scale=2 * sensitivity**2 * np.log(1.25 / delta) / epsilon**2)
    
    elif mechanism == "exponential":
        # this query can only change by 1 if an instance is omitted so l1-sensitivity = 1
        sensitivity = 1
        
        # np.arange is [start, stop) so + 1 for entire possible range
        possible_values = np.arange(0, sens_dataset.loc[dataset[pd.eval(rule, engine='python')].index].value_counts().to_numpy().sum() + 1)
        
        # the utility is higher when the value is closer to the actual value
        utility_scores = np.array([no_mechanism - abs(no_mechanism - value) for value in possible_values]) / 100
        probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in utility_scores]
        
        # normalize probabilties to sum to 1
        probabilities /= np.linalg.norm(probabilities, ord=1)
        return np.random.choice(possible_values, p=probabilities)

    # if no mechanism is given, return the unprivatised cocunt
    return no_mechanism


In [21]:
def statistical_parity(y_pred, sens_dataset):
    """Calculates Statistical Parity Ratio using the predictions and the actual sensitive feature values. 
    
    Args:
        y_pred: The predictions, should be of same size as sens_dataset.
        sens_dataset: The Series with the sensitive attributes.
        
    Returns:
        The true statistical parity ratio.
        """
    accept_rates = []
    
    for sens_attr in sorted(sens_dataset.unique()):
        accept_rates.append(np.sum((sens_dataset == sens_attr) & y_pred) / np.sum(sens_dataset == sens_attr))
        
    return min(accept_rates) / max(accept_rates)


def estimate_sp(pos_ruleset, dataset, sens_dataset, S, mechanism, epsilon, delta=0.001):
    """Returns the estimated Statistical Parity of a tree for a privacy mechanism. The PAFER algorithm. 
    
    Args:
        pos_ruleset: A list of rules that classify favorably in the tree. This is the representation of the
        (relevant parts of the) tree. 
        dataset: The DataFrame that the developers own that does not contain sensitive feature values.
        sens_dataset: The Series that contains the sensitive features, which the developers do not own.
        S: The set/list of sensitive attributes, should all be in the sens_dataset attribute.
        mechanism: The mechanism with which to privatise the query answers. 
        epsilon: The privacy budget for the privacy mechanism. Should be larger than 0.
        delta: The privacy margin. Ignored when mechanism is either laplacian or gaussian. Should be in (0, 1].
        
    Returns:
        The statistical parity ratio for the specified pos_ruleset. 
        """
    
    poscounts_per_si = np.zeros(len(S))
    
    # the variable name of the current dataset is inferred from the ruleset
    datasetname = str(pos_ruleset[0].split('[')[0])[1:]
    
    # the base rule is a rule that includes all individuals, i.e. the condition is a tautology
    # in this case we select all rows that have a value that is in the set of possible values of the first column
    base_rule = f"({datasetname}[{datasetname}.columns[0]].isin({datasetname}[{datasetname}.columns[0]].unique()))"
    
    # query the size of each sensitive attribute in the dataset
    total_per_si = [oracle(dataset, sens_dataset, base_rule, s_i, mechanism, 0.5 * epsilon, delta) for s_i in S]
    
    # replace each invalid value with balanced totals
    for i, tot in enumerate(total_per_si):
        if tot < 0 or tot > len(sens_dataset):
            total_per_si[i] = (1 / len(S)) * len(sens_dataset)
        
    total_per_si = np.array(total_per_si)
    
    for rule in pos_ruleset:
        # for each rule we find the distribution of sensitive attributes
        rule_counts = np.zeros(len(S))
        rule_total = len(sens_dataset[pd.eval(rule)])
        
        for i, s_i in enumerate(S):
            # because the queries are disjoint, epsilon remains equal across queries
            answer = round(oracle(dataset, sens_dataset, rule, s_i, mechanism, 0.5 * epsilon, delta))

            # if invalid answers from query: replace with balanced node value
            if answer < 0 or answer > len(sens_dataset):
                answer = (1 / len(S)) * rule_total

            rule_counts[i] += answer
        
        # the distribution for the current rule is added to the total
        poscounts_per_si += rule_counts
    
    # calculate and return sp
    accept_rates = poscounts_per_si / total_per_si
    return np.min(accept_rates) / np.max(accept_rates)


## Tree construction pipeline

In [37]:
def find_best_tree(dataset, dataset_labels, minleaf=1, ccp_alpha=0.0):
    """Train a tree for balanced accuracy performance using grid search. 
    
    dataset: The training data (without sensitive attributes!).
    dataset_labels: The true outcomes for the prediction task.
    minleaf: The tree construction parameter denoting the minimum number of instances in a leaf node.
        minleaf should be in (0, 1]. 
        
    Returns: 
        The best performing decision tree."""
    
    # no random_state because we want a different tree each run
    tree = DecisionTreeClassifier()

    parameter_grid = {"criterion":["entropy", "gini"],
                      "max_features":["sqrt", "log2"], 
                      "min_samples_leaf":[minleaf], "ccp_alpha": [ccp_alpha]}
    
    # train and return the best tree
    tree_cv = GridSearchCV(tree, param_grid=parameter_grid, scoring='balanced_accuracy', n_jobs=2, cv=3, verbose=0)
    tree_cv.fit(dataset, dataset_labels)
    best_tree = tree_cv.best_estimator_
    return best_tree

best_tree = find_best_tree(taiwan_train_X, taiwan_train_y, ccp_alpha=0.05)

In [29]:
# taken from: https://stackoverflow.com/a/51398390
def is_leaf(inner_tree, index):
    # check whether node is leaf node
    return (inner_tree.children_left[index] == TREE_LEAF and 
            inner_tree.children_right[index] == TREE_LEAF)

def prune_index(inner_tree, decisions, index=0):
    # start pruning from the bottom - if we start from the top, we might miss
    # nodes that become leaves during pruning
    if not is_leaf(inner_tree, inner_tree.children_left[index]):
        prune_index(inner_tree, decisions, inner_tree.children_left[index])
    if not is_leaf(inner_tree, inner_tree.children_right[index]):
        prune_index(inner_tree, decisions, inner_tree.children_right[index])

    # prune children if both children are leaves now and make the same decision
    if (is_leaf(inner_tree, inner_tree.children_left[index]) and
        is_leaf(inner_tree, inner_tree.children_right[index]) and
        (decisions[index] == decisions[inner_tree.children_left[index]]) and 
        (decisions[index] == decisions[inner_tree.children_right[index]])):
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF

def prune_duplicate_leaves(mdl):
    # Remove leaves if all siblings make the same decision
    decisions = mdl.tree_.value.argmax(axis=2).flatten().tolist() # Decision for each node
    prune_index(mdl.tree_, decisions)
    
# pruning happens in-place
prune_duplicate_leaves(best_tree)

In [30]:
def positive_rules (tree, rules):
    """From the extracted rules, return those that have a favorable classification. 

    Arg:
        tree: The tree classification object from which the rules are extracted. 
        rules: Dict of which the values are rule strings.

    Returns:
        A list of all the rules that classify favorably"""

    # only those rules are added for which the majority of individuals in the node is at index 1, i.e. max
    # index 1 corresponds to class 1 which we ensured was the favorable outcome
    return [rule for node_id, rule in rules.items() if np.argmax(tree.tree_.value[node_id][0])]


In [31]:
# taken from: https://stackoverflow.com/a/56427596
def extract_pos_rules(tree, dataset, datasetname):
    n_nodes = tree.tree_.node_count
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    feature = tree.tree_.feature
    threshold = tree.tree_.threshold

    def find_path(node_numb, path, x):
        path.append(node_numb)
        if node_numb == x:
            return True
        left = False
        right = False
        if (children_left[node_numb] !=-1):
            left = find_path(children_left[node_numb], path, x)
        if (children_right[node_numb] !=-1):
            right = find_path(children_right[node_numb], path, x)
        if left or right :
            return True
        path.remove(node_numb)
        return False


    def get_rule(datasetname, path, column_names):
        mask = '('
        for index, node in enumerate(path):
            # check if we are not in the leaf
            if index!=len(path)-1:
                # under or over the threshold?
                if (children_left[node] == path[index+1]):
                    mask += f"{datasetname}['{column_names[feature[node]]}']<= {threshold[node]}\t "
                else:
                    mask += f"{datasetname}['{column_names[feature[node]]}']> {threshold[node]} \t "

        # insert the & at the right places
        mask = mask.replace("\t", "&", mask.count("\t") - 1)
        mask = mask.replace("\t", "")
        mask += ")"
        return mask
    
    # Leaves
    leave_id = tree.apply(dataset)

    paths = {}
    for leaf in np.unique(leave_id):
        path_leaf = []
        find_path(0, path_leaf, leaf)
        paths[leaf] = np.unique(np.sort(path_leaf))

    rules = {}
    for key in paths:
        rules[key] = get_rule(datasetname, paths[key], [name for name in dataset.columns])
        
    return positive_rules(tree, rules)
        
extract_pos_rules(best_tree, taiwan_train_X, "taiwan_train_X")

["(taiwan_train_X['PAY_2']> 1.5  )"]

## Experiments

In [32]:
def bootstrap(dataset, dataset_labels, sens_dataset):
    """A bootstrapping function that helps to diversify the tree generating process."""
    indices = np.random.choice(dataset.index, size=len(dataset.index))
    
    return dataset.iloc[indices], dataset_labels.iloc[indices], sens_dataset.iloc[indices]



In [33]:
def experiment(trainset, sens_trainset, trainsetname, trainset_labels, testset, sens_testset, testsetname, 
               testset_labels, epsilons=[0.05, 0.1, 0.15, 0.2, 0.25], minleaf=1, ccp_alpha=0.0, runs=5, combined=False):
    """Performs an experiment as described in the paper.
    
    trainset: The DataFrame containing the training instances.
    sens_trainset: The Series containing the sensitive attribute values for the trainset.
    trainsetname: The variable name of the trainset. Required because of rule evaluation.
    trainset_labels: The true outcomes for the prediction task for the trainset.
    testset: The DataFrame containing the test instances.
    sens_testset: The Series containing the sensitive attribute values for the testset.
    testsetname: The variable name of the testset. Required because of rule evaluation.
    testset_labels: The true outcomes for the prediction task for the testset.
    epsilons: The different privacy budgets to try. 
        epsilons must be a list.
    minleaf: The tree construction parameter denoting the minimum number of instances in a leaf node.
        minleaf should be in (0, 1].
    runs: The number of runs to average over. Advised to be quite high (e.g. 50) to compensate for noise.
    combined: Whether to combine all positive rules into one query. 
    
    Returns:
        The true SP of the tree and the estimated SP."""
    
    
    tree_sps = np.zeros((runs, len(epsilons)))
    tree_depths = np.zeros((runs, len(epsilons)))
    estimated_sps = np.zeros((runs, len(epsilons)))
    for i in range(runs):
        ruleset = []
        
        # keep boostrapping until we find a ruleset that has at least one positive rule
        while ruleset == [] or ruleset == ['()']:
            # sample with replacement
            dataset, dataset_labels, sens_dataset = bootstrap(trainset, trainset_labels, sens_trainset)
            
            # build tree 
            best_tree = find_best_tree(trainset, trainset_labels, minleaf, ccp_alpha)
        
            # extract positive rules
            prune_duplicate_leaves(best_tree)
            ruleset = extract_pos_rules(best_tree, trainset, testsetname)
        
        if combined:
            ruleset = [" | ".join(rule for rule in ruleset)]

        # calculate true SP
        tree_sps[i] = statistical_parity(best_tree.predict(testset), sens_testset)
        tree_depths[i] = best_tree.tree_.max_depth
        
        # apply PAFER
        for j, epsilon in enumerate(epsilons):
            estimated_sps[i, j] = estimate_sp(ruleset, testset, sens_testset, sorted(sens_testset.unique()), mechanism='laplacian', epsilon=epsilon)
        
    return tree_sps, tree_depths, estimated_sps
        

## MINLEAF EXPERIMENTS

In [39]:
# script to run the experiments
# TODO: current datastructures are not optimal or intuitive so could be helpful to streamline (for plotting)
minleafs = np.linspace(0.2, 0.001, 80)
runs = 50

# storage for results
tree_sps = []
tree_depths = []
estimated_sps = []
for minleaf in tqdm(minleafs):
    t_sps, t_depths, e_sps = experiment(taiwan_train_X, taiwan_train_sex, "taiwan_train_X", taiwan_train_y, 
                                         taiwan_test_X, taiwan_test_sex, "taiwan_test_X", taiwan_test_y, minleaf=minleaf, runs=runs)
    
    tree_sps.append(t_sps)
    tree_depths.append(t_depths)
    estimated_sps.append(e_sps)
    

100%|██████████████████████████████████████████| 10/10 [56:04<00:00, 336.41s/it]


In [ ]:
tree_sps = np.array(tree_sps)
tree_depths = np.array(tree_depths)
estimated_sps = np.array(estimated_sps)

for arr, name in zip([tree_sps, tree_depths, estimated_sps], ["tree_sps", "tree_depths", "estimated_sps"]):
    with open(f"taiwan-sex-minleaf-{name}", "wb") as f:
        np.save(f, arr)

In [ ]:
# script to run the experiments
ccp_alphas = np.linspace(0.05, 0.001, 80)
runs = 50

# storage for results
tree_sps = []
tree_depths = []
estimated_sps = []
for ccp_alpha in tqdm(ccp_alphas):
    t_sps, t_depths, e_sps = experiment(taiwan_train_X, taiwan_train_sex, "taiwan_train_X", taiwan_train_y, 
                                         taiwan_test_X, taiwan_test_sex, "taiwan_test_X", taiwan_test_y, ccp_alpha=ccp_alpha, runs=runs)
    
    tree_sps.append(t_sps)
    tree_depths.append(t_depths)
    estimated_sps.append(e_sps)
    

In [ ]:
tree_sps = np.array(tree_sps)
tree_depths = np.array(tree_depths)
estimated_sps = np.array(estimated_sps)

for arr, name in zip([tree_sps, tree_depths, estimated_sps], ["tree_sps", "tree_depths", "estimated_sps"]):
    with open(f"taiwan-sex-ccp-{name}", "wb") as f:
        np.save(f, arr)